In [1]:
import torch,sys,os

from tqdm import tqdm

from torch import nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms

# Self reproduce

In [ ]:
from typing import Optional,Callable

class GoogLeNet_v1(nn.Module):
    def __init__(self,num_classes=1000,init_weights=False):
        super(GoogLeNet_v1,self).__init__()

        self.Conv = nn.Sequential(
            # 使用7x7的卷积核（滑动步长2，padding为3），64通道，输出为112x112x64，卷积后进行ReLU操作
            nn.Conv2d(kernel_size=7,stride=2,padding=3,out_channels=64,in_channels=3), 
            nn.ReLU(),

            # 经过3x3的max pooling（步长为2），输出为((112 - 3+1)/2)+1=56，即56x56x64，再进行ReLU操作
            nn.MaxPool2d(stride=2,kernel_size=3),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, k=2, alpha=1e-4, beta=0.75),

            # 使用3x3的卷积核（滑动步长为1，padding为1），192通道，输出为56x56x192，卷积后进行ReLU操作

            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=1),
            nn.LocalResponseNorm(size=5, k=2, alpha=1e-4, beta=0.75),
            nn.Conv2d(in_channels=64,out_channels=192,kernel_size=3,stride=1),
            nn.LocalResponseNorm(size=5, k=2, alpha=1e-4, beta=0.75),
            nn.ReLU(),
            # 经过3x3的max pooling（步长为2），输出为((56 - 3+1)/2)+1=28，即28x28x192，再进行ReLU操作
            nn.MaxPool2d(kernel_size=3,stride=2),
            nn.ReLU()
        )

        self.Inception_3a = Inception(in_channels=64,ch1x1=1)

    def forward(self,x):
        x= self.Conv(x)
        y
        return y


class Inception(nn.Module):
    def __init__(self,
        in_channels: int,
        ch1x1: int,
        ch3x3red: int,
        ch3x3: int,
        ch5x5red: int,
        ch5x5: int,
        pool_proj: int,
        conv_block: Optional[Callable[..., nn.Module]] = None,) -> None:
        super().__init__()

        



# Pytorch Version

Using cuda device


 # STL10 dataset

In [3]:
transform=transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5,),(0.5,0.5,0.5)),
	transforms.Resize([224, 224])
    ]
)

# Download training data from open datasets.
train_set = datasets.STL10(
    root="~/data/STL10/",
    split ='train',
    download=True,
    transform=transform, # transform,
)
trainloader=torch.utils.data.DataLoader(
	train_set,
	batch_size=60,
	shuffle=True,
	pin_memory=True,
    num_workers=8
	)


# Download test data from open datasets.
test_set = datasets.STL10(
    root="~/data/STL10/",
    split ='test',
    download=True,
    transform=transform, # transform,
)
testloader=torch.utils.data.DataLoader(
	test_set,
	batch_size=60,
	shuffle=False,
    pin_memory=True,
    num_workers=8
	)

# test_data_iter=iter(testloader)
# test_image,test_label=test_data_iter.next()
test_num  = len(test_set)
train_steps = len(trainloader)

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


_IncompatibleKeys(missing_keys=['Classifier.0.weight', 'Classifier.0.bias', 'Classifier.3.weight', 'Classifier.3.bias', 'Classifier.6.weight', 'Classifier.6.bias'], unexpected_keys=['classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias', 'classifier.6.weight', 'classifier.6.bias'])

# loss and optimize

In [5]:
# 定义一个损失函数
loss_fn = nn.CrossEntropyLoss()

# 定义一个优化器
# optimizer = torch.optim.Adam(model.parameters(),lr=0.005)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
# optimizer = torch.optim.SGD(model.parameters(),lr=0.001)

epochs = 40

save_path= './VGG16.pth'
best_acc = 0.0


# Transfer learning

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# from torchvision import models
# pretrained_VGG16 = models.vgg16(pretrained=True)
# torch.save(pretrained_VGG16.state_dict(), save_path)

model = VGG16(num_classes=10, init_weights=True).to(device)
model.load_state_dict(torch.load('./VGG16.pth'),strict=False)

# Train

In [7]:
for epoch in range(epochs):
        # train
        model.train()
        running_loss = 0.0
        train_bar = tqdm(trainloader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            outputs = model(images.to(device))
            loss = loss_fn(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,epochs,loss)

        # validate
        model.eval()
        acc = 0.0  # accumulate accurate number / epoch
        with torch.no_grad():
            val_bar = tqdm(testloader, file=sys.stdout) # show progress
            for val_data in val_bar:
                val_images, val_labels = val_data
                outputs = model(val_images.to(device))
                predict_y = torch.max(outputs, dim=1)[1]
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

        val_accurate = acc / test_num
        print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, val_accurate))

        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(model.state_dict(), save_path)

print('Finished Training')

100%|██████████| 134/134 [00:38<00:00,  3.49it/s]
[epoch 1] train_loss: 2.189  val_accuracy: 0.696
100%|██████████| 134/134 [00:40<00:00,  3.27it/s]
[epoch 2] train_loss: 1.365  val_accuracy: 0.832
100%|██████████| 134/134 [00:40<00:00,  3.33it/s]
[epoch 3] train_loss: 0.854  val_accuracy: 0.872
100%|██████████| 134/134 [00:39<00:00,  3.38it/s]
[epoch 4] train_loss: 0.623  val_accuracy: 0.902
100%|██████████| 134/134 [00:41<00:00,  3.27it/s]
[epoch 5] train_loss: 0.502  val_accuracy: 0.913
100%|██████████| 134/134 [00:40<00:00,  3.30it/s]
[epoch 6] train_loss: 0.390  val_accuracy: 0.920
100%|██████████| 134/134 [00:40<00:00,  3.31it/s]
[epoch 7] train_loss: 0.339  val_accuracy: 0.924
100%|██████████| 134/134 [00:40<00:00,  3.30it/s]
[epoch 8] train_loss: 0.281  val_accuracy: 0.925
100%|██████████| 134/134 [00:40<00:00,  3.33it/s]
[epoch 9] train_loss: 0.267  val_accuracy: 0.930
100%|██████████| 134/134 [00:40<00:00,  3.33it/s]
[epoch 10] train_loss: 0.233  val_accuracy: 0.933
100%|████

KeyboardInterrupt: ignored